In [1]:
import json
import numpy as np
import csv

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import random
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score
import nltk

import re
import pandas as pd
import bs4
import requests
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')

from spacy.matcher import Matcher 
from spacy.tokens import Span 

import networkx as nx

import matplotlib.pyplot as plt
from tqdm import tqdm


from sklearn import preprocessing

pd.set_option('display.max_colwidth', 200)
%matplotlib inline


In [2]:
# import relevant LMs package

import transformers
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, DataLoader, random_split
from transformers import BertForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import f1_score, accuracy_score


In [3]:
# cheak to use gpu or cpu
if torch.cuda.is_available():
    print("Use gpu")
    device = torch.device("cuda")
else:
    print("Use cpu")
    device = torch.device("cpu")

Use gpu


In [4]:
device = "cuda:0"
torch.cuda.empty_cache()
# torch.cuda.reset_max_memory_allocated()

In [5]:
path='C:\\Users\\wz\\Desktop\\context-aware-embedding-master\\Resources\\code\\data\\DBpedia_query\\train\\DBpedia_query_train.csv'
DBpedia = pd.read_csv(path)
DBpedia

,Unnamed: 0,query,response
0,86145,Mystery Mile previousWork,The Crime at Black Dudley
1,1527443,Mohamed Sylla (footballer born 1971) birthPlace,Conakry
2,9886997,Old-time music stylisticOrigin,Music of the United Kingdom
3,10007780,Blood on Ice artist,Bathory (band)
4,209861,Richard D. James Album subsequentWork,Come to Daddy
...,...,...,...
79995,4964299,Conidiosporomyces order,Tilletiales
79996,6679886,The Fully Down hometown,Canada
79997,8233302,Marco Scutaro team,Oakland Athletics
79998,4936904,Geitodoris bonosi order,Heterobranchia


In [6]:
# next token prediction
from transformers import AutoModelForCausalLM, AutoTokenizer


print("\nBegin next-word using HF GPT-2 demo ")

toker = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")



Begin next-word using HF GPT-2 demo 


KeyboardInterrupt: 

In [ ]:
seq = "connie own"
print("\nInput sequence: ")
print(seq)

inpts = toker(seq, return_tensors="pt")
print("\nTokenized input data structure: ")
print(inpts)

inpt_ids = inpts["input_ids"]  # just IDS, no attn mask
print("\nToken IDs and their words: ")
for id in inpt_ids[0]:
    word = toker.decode(id)
    print(id, word)

    
with torch.no_grad():
    logits = model(**inpts).logits[:, -1, :]

print("\nAll logits for next word: ")
print(logits)
print(logits.shape)

pred_id = torch.argmax(logits).item()
print("\nPredicted token ID of next word: ")
print(pred_id)

print("testing ", torch.argmax(logits))

# pred_id = i.item()
# pred_word = toker.decode(pred_id)
# print("\nPredicted next word for sequence: ")
# print(pred_word)

# for i in logits[0]:
#     print(i.type())
#     # pred_id = i.item()
#     pred_id = i.type(torch.LongTensor)
    
#     pred_id = pred_id.type(torch.FloatTensor)
#     print(pred_id)
#     pred_word = toker.decode(pred_id)
#     print("\nPredicted next word for sequence: ")
#     print(pred_word)


In [ ]:
from  transformers import GPT2TokenizerFast, GPT2LMHeadModel

base_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
vocabulary = base_tokenizer.get_vocab()
vocabulary['University']

text = 'University of Wisconsin%E2%80%93Madison'
sub = 'University'
text_ids = base_tokenizer.encode(sub, return_tensors = 'pt')
text_ids.item()

In [7]:
# import torch

# from  transformers import GPT2TokenizerFast, GPT2LMHeadModel
# from torch.nn import CrossEntropyLoss

# model = GPT2LMHeadModel.from_pretrained("gpt2")
# model.eval()
# tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

# ### We calculate the hidden_states and the past of the common left part of the sentence
# past = "I like sitting in my new chair and"
# past_tokenize_input = tokenizer.tokenize(past)
# past_tensor_input = torch.tensor([tokenizer.convert_tokens_to_ids(past_tokenize_input)])

# past_last_hidden_state, past = model.transformer(past_tensor_input)

# print(past_last_hidden_state)

# def score(sentence, past, past_last_hidden_state, past_tensor_input):
#     tokenize_input = tokenizer.tokenize(sentence, )
#     tensor_input = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_input)])

#     ###the following code is slightly modified from https://github.com/huggingface/transformers/blob/09a2f40684f77e62d0fd8485fe9d2d610390453f/src/transformers/modeling_gpt2.py#L604
#     ###now we calculate the right part of the sentence with the already calculated past
#     transformer_outputs = model.transformer(
#             tensor_input,
#             attention_mask=None,
#             token_type_ids=None,
#             position_ids=None,
#             head_mask=None,
#             inputs_embeds=None,
#             use_cache=None,
#             output_attentions=None,
#             output_hidden_states=None,
#         )
    
#     # print(past_last_hidden_state)
        
#     ###and concatenate the output of with the hidden_state of the left part of the sentence
#     hidden_states = torch.cat((past_last_hidden_state, transformer_outputs[0]), dim=1)
    
#     ###the following part is exactly the same as https://github.com/huggingface/transformers/blob/09a2f40684f77e62d0fd8485fe9d2d610390453f/src/transformers/modeling_gpt2.py#L604
#     lm_logits = model.lm_head(hidden_states)

#     labels_input = torch.cat((past_tensor_input, tensor_input), dim=1)

#     # Shift so that tokens < n predict n
#     shift_logits = lm_logits[..., :-1, :].contiguous()
#     shift_labels = labels_input[..., 1:].contiguous()
#     # Flatten the tokens
#     loss_fct = CrossEntropyLoss()
#     loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
#     return -loss.item()

# candidates = ["watch", "run", "think", "apple", "light"]

# sent_template = " {} about life"

# print({candidate: score((sent_template.format(candidate)), past, past_last_hidden_state, past_tensor_input) for candidate in candidates})

In [8]:
# get a subgraph that include the true answer with input row
def get_label(row, n):
    text = DBpedia.loc[3,"query"]
    true_answer = DBpedia.loc[3,"response"]
    DBpedia_s = DBpedia.sample(n-1)
    DBpedia_row = DBpedia.iloc[[row]]
    DBpedia_sub = pd.concat([DBpedia_s,DBpedia_row], axis=0)
    # get labels
    le = preprocessing.LabelEncoder()
    all_response = DBpedia_sub.loc[:,'response'].values
    le.fit(all_response)
    label_value = le.transform(all_response)
    # form a dict labels
    labels = dict()
    for i in range(len(all_response)):
        labels[all_response[i]] = label_value[i]
    # print(len(all_response))
    return text, true_answer, all_response, label_value, labels

In [9]:
# all_response = all_response[0:100]

In [10]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.eval()
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

def next_word_prediction(sentence):
    tokenize_input = tokenizer.tokenize(sentence)
    tensor_input = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_input)])
    loss = model(tensor_input, labels=tensor_input)
    return -loss[0].item()

def outcome(candidates, sentence, true_answer):
    # candidates = all_response
    sent_template = sentence + " {}"
    # print({candidate: next_word_prediction(sent_template.format(candidate)) for candidate in candidates})
    response_prob = {candidate: next_word_prediction(sent_template.format(candidate)) for candidate in candidates}
    max_key = max(response_prob, key=lambda k: response_prob[k])
    # loss from true response 
    loss = response_prob[max_key] - response_prob[true_answer]
    return max_key, loss

In [11]:
def next_word_GPT2(row, n):
    # print( get_label(3,100))
    text, true_answer, all_response, label_value, labels = get_label(3,100)
    max_key, loss = outcome(all_response,text,true_answer)
    return max_key, loss

In [12]:
next_word_GPT2(10,100)

('Ron Jones (composer)', 1.5008091926574707)

In [13]:
# # get a bound for loss for 3 trail
# for it in range(3):
#     array = []
#     for i in tqdm(range(len(DBpedia))):
#         array.append(next_word_GPT2(i,100))
#     print("The median is ", np.median(array))

In [14]:
# np.median(array)

In [15]:
%load_ext autoreload
%autoreload 2

import torch
import wandb
import time
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
from random import choices
import matplotlib.pyplot as plt
tqdm.pandas()

from datasets import load_dataset

from transformers import GPT2Tokenizer
from transformers import AutoModelForSequenceClassification, AutoTokenizer

from trl.gpt2 import GPT2HeadWithValueModel, respond_to_batch
from trl.ppo import PPOTrainer
from trl.core import build_bert_batch_from_txt

config = {
    "lm_name": "lvwerra/gpt2",
    "ref_lm_name": "lvwerra/gpt2",
    "tk_name": "gpt2",
    "steps": 6400,
    "batch_size": 64,
    "forward_batch_size": 8,
    "ppo_epochs": 4,   
    "txt_in_len": 5,
    "txt_out_len": 15,
    "lr": 1.41e-5,
    "init_kl_coef":0.2,
    "target": 6,
    "horizon":10000,
    "gamma":1,
    "lam":0.95,
    "cliprange": .2,
    "cliprange_value":.2,
    "vf_coef":.1, 
    "seed": 1,
}

np.random.seed(config['seed'])

In [16]:
# trails
wandb.init(name='trails #15', project='gpt2-query', config=config)

wandb: Currently logged in as: wz_ryan. Use `wandb login --relogin` to force relogin


In [17]:
# load models
gpt2_model = GPT2HeadWithValueModel.from_pretrained("gpt2")
gpt2_model_ref = GPT2HeadWithValueModel.from_pretrained("gpt2")
gpt2_tokenizer = GPT2Tokenizer.from_pretrained(config['tk_name'])

Some weights of GPT2HeadWithValueModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias', 'v_head.summary.bias', 'lm_head.weight', 'v_head.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of GPT2HeadWithValueModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias',

In [18]:
_ = gpt2_model.to(device)
_ = gpt2_model_ref.to(device)

In [19]:
wandb.watch(gpt2_model, log='all')

[]

In [20]:
# bert classifier
device = "cuda:0"
sentiment_model = AutoModelForSequenceClassification.from_pretrained("lvwerra/distilbert-imdb")
sentiment_tokenizer = AutoTokenizer.from_pretrained("lvwerra/distilbert-imdb")


In [21]:
# rename
df = DBpedia.copy()
# ctrl_str = ['[false]', '[true]']
# only going for factual control
ctrl_str = ['[true]']
ctrl_tokens = dict((s, gpt2_tokenizer.encode(s, return_tensors="pt").squeeze().to(device)) for s in ctrl_str)

In [22]:
# special token
df['tokens'] = df['query'].apply(lambda x: gpt2_tokenizer.encode(' '+str(x), return_tensors="pt").to(device))
df['tokens'] = df['tokens'].apply(lambda x: x[0])
df['tokens'] = df['tokens'].apply(lambda x: x[:2])

# dropping invalid length in token column
drop_row_index = []
for i in range(1000):

    if len(df['tokens'].loc[i]) < 2:
        #df['tokens'].loc[i] = df['tokens'].loc[0].cuda().detach().cpu().clone().numpy()
        drop_row_index.append(i)
print(drop_row_index)
df = df.drop(df.index[drop_row_index])

[]


In [23]:
def pos_logit_to_reward(logit, task):
    """
    Take the positive sentiment logit and scale it for the task.
        task [negative]: reward = -logit
        task [positive]: reward = logit
    """
    for i in range(len(logit)):
        if task[i]=='[false]':
            logit[i] = -logit[i]
        elif task[i]=='[true]':
            pass
        else:
            raise ValueError('task has to be in [0, 1, 2]!')
    return logit

In [24]:
### KG check to modify reward value 

# given a range i.e. i*fbs to (i+1)*fbs
# should output a tensor([ 0, 0,  1, 1], device='cuda:0')

# pred is the reponse in batch_response
# query is the query_list

def KG_check(lower, upper, threshold=4):
    factuals = []
    response = []
    for i in range(lower,upper):
        key, loss = next_word_GPT2(i,100)
        factual = 0
        if loss <= threshold:
            factual =1
        factuals.append(factual)
        response.append(key)
    # add variant 
    tensor_array = [i + random.random() for i in factuals ]
    return torch.cuda.LongTensor(tensor_array), response


In [25]:
KG_check(3, 7, threshold=4)

C:\Users\wz\AppData\Local\Temp\ipykernel_11504\2530387475.py:21: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.cuda.LongTensor(tensor_array), response


(tensor([1, 1, 1, 1], device='cuda:0'),
 ['The Legend of Zelda: The Wind Waker',
  'Kiyoshi Kobayashi',
  'Arminia Bielefeld',
  'Hitomi Takahashi (singer)'])

In [26]:
# batch_size: 32
# forward_batch_size: 4 

ppo_trainer = PPOTrainer(gpt2_model, gpt2_model_ref, **config)
fbs = config['forward_batch_size']

for epoch in tqdm(range(int(np.ceil(config["steps"]/config['batch_size'])))):
    torch.cuda.empty_cache()
    logs = dict()
    game_data = dict()
    timing = dict()
    t0 = time.time()
    
    #### get a batch from the dataset and annotate tasks
    df_batch = df.sample(config['batch_size'])
    task_list = choices(ctrl_str, k=config['batch_size'])
    task_tensors = torch.stack([ctrl_tokens[t] for t in task_list])
    query_list = df_batch['query'].tolist()
    game_data['query'] = [q for q in zip(query_list)]
    # game_data['query'] = [t+q for t,q in zip(task_list, query_list)]
    
    
    #print(task_tensors)
    query_tensors = torch.stack((df_batch['tokens'].tolist()))
    #print("before ",query_tensors)
    query_tensors = torch.cat((task_tensors, query_tensors), axis=1)

    
    #### get response from gpt2
    t = time.time()
    total_length = config['txt_in_len']+config['txt_out_len']
    response_tensors_array = []
    for i in range(int(config['batch_size']/fbs)):
        response  = respond_to_batch(gpt2_model, query_tensors[i*fbs:(i+1)*fbs],
                                     txt_len=config['txt_out_len'])
        response_tensors_array.append(response)
    response_tensors = torch.cat(response_tensors_array)
#         #print(int(config['batch_size']/fbs))
#         # 8 loops in total
#         # 0-4, 5-8, 9-12...
        
#         # this is troublesome 
#         # error: probability tensor contains either `inf`, `nan` or element < 0
#         #  next_token = torch.multinomial(probs, num_samples=1).squeeze(1) in gpt2.py function respond_to_batch
        
#         # response  = respond_to_batch(gpt2_model, query_tensors[i*fbs:(i+1)*fbs],txt_len=config['txt_out_len'])
#         # we want a response of size [i*fbs:(i+1)*fbs]*1
#         response_tensors = []
#         for i in range(i*fbs, (i+1)*fbs):
            
#             seq = query_list[i]
#             inpts = toker(seq, return_tensors="pt")
#             inpt_ids = inpts["input_ids"]  # just IDS, no attn mask
            
#             key, loss = next_word_GPT2(i,100)
            
#             pred_id = base_tokenizer.encode(key, return_tensors = 'pt')
#             # now it force response list into one tensor
#             response = (pred_id[0])[1] # todo change it  should be one id not a list
#             # print(response)
#             response_tensors.append(response.item())
#         response_tensors = torch.cuda.LongTensor(response_tensors)
#         response_tensors_array.append(response_tensors)
   
#     # response_tensors.size = 32 * 1  
#     response_tensors = torch.cat(response_tensors_array,axis = -1 )
#     response_tensors  = torch.reshape(response_tensors, (-1, 1)) # force size of response_tensors.size = 32 * 1  rather that 32 length list
#     # print(response_tensors)
#     # print([gpt2_tokenizer.decode(response_tensors[i, :]) for i in range(config['batch_size'])])
#     # print(response_tensors)
    batch_response = [gpt2_tokenizer.decode(response_tensors[i, :]) for i in range(config['batch_size'])] # list of length 32
    #game_data['response'] = batch_response
    #timing['time/get_response'] = time.time()-t
    #print( game_data['response'])

    #### tokenize text for sentiment analysis
#     t = time.time()
#     texts = [q + r for q,r in zip(query_list, game_data['response'])]
#     sentiment_inputs, attention_masks = build_bert_batch_from_txt(texts, sentiment_tokenizer, device)    
#     timing['time/build_input_sentiment_unused'] = time.time()-t
        
    #### get sentiment score
    t = time.time()
    pos_logits = []
    batch_response_new = []
    for i in range(int(config['batch_size']/fbs)):
        device = torch.device("cuda")
        # modify the rewards based on the KG
        res, response = KG_check(i*fbs, (i+1)*fbs)
        pos_logits.append(res)
        batch_response_new = np.concatenate((batch_response_new, response))

    
    game_data['response'] = batch_response_new
    timing['time/get_response'] = time.time()-t
    
    t = time.time()
    texts = [q + r for q,r in zip(query_list, game_data['response'])]
    sentiment_inputs, attention_masks = build_bert_batch_from_txt(texts, sentiment_tokenizer, device)    
    timing['time/build_input_sentiment_unused'] = time.time()-t
    
    rewards = pos_logit_to_reward(torch.cat(pos_logits), task_list)
    timing['time/get_factual_preds'] = time.time()-t


    #### Run PPO training 
    t = time.time()
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    
    # enforce nan to zero for wandb to run
    stats = np.nan_to_num(stats)
    # print(stats)
    timing['time/optimization'] = time.time()-t
     
    #### Log everything
    timing['time/epoch'] = time.time()-t0
    table_rows = [list(r) for r in zip(game_data['query'], game_data['response'], rewards.cpu().tolist())]
    logs.update({'game_log':wandb.Table(
        columns=['query', 'response', 'reward'],
        rows=table_rows)})
    logs.update(timing)
    # comment to remove the nan issues
    logs.update(stats)
    
    # only support for float tensor
    rewards = rewards.type(torch.FloatTensor)
    
    logs['env/reward_mean'] = torch.mean((rewards)).cpu().numpy()
    logs['env/reward_std'] = torch.std(rewards).cpu().numpy()
    logs['env/reward_dist'] = rewards.cpu().numpy()
    for ctrl_s in ctrl_str:
        key = 'env/reward_'+ctrl_s.strip('[]')
        logs[key] = np.mean([r for r, t in zip(logs['env/reward_dist'], task_list) if t==ctrl_s])
#     WandbCallback(log=None)
    wandb.log(logs)

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\wz\AppData\Local\Temp\ipykernel_11504\2530387475.py:21: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.cuda.LongTensor(tensor_array), response
C:\Users\wz\AppData\Local\Temp\ipykernel_11504\2530387475.py:21: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.cuda.LongTensor(tensor_array), response
C:\Users\wz\AppData\Local\Temp\ipykernel_11504\2530387475.py:21: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.cuda.LongTensor(tensor_array), response
C:

C:\Users\wz\AppData\Local\Temp\ipykernel_11504\2530387475.py:21: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.cuda.LongTensor(tensor_array), response
C:\Users\wz\AppData\Local\Temp\ipykernel_11504\2530387475.py:21: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.cuda.LongTensor(tensor_array), response
C:\Users\wz\AppData\Local\Temp\ipykernel_11504\2530387475.py:21: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.cuda.LongTensor(tensor_array), response
C:\Users\wz\AppData\Local\Temp\ipykernel_11504\2530387475.py:21: DeprecationWarning: an integer is required (got type flo

C:\Users\wz\AppData\Local\Temp\ipykernel_11504\2530387475.py:21: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.cuda.LongTensor(tensor_array), response
C:\Users\wz\AppData\Local\Temp\ipykernel_11504\2530387475.py:21: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.cuda.LongTensor(tensor_array), response
  7%|█████▌                                                                         | 7/100 [35:31<7:51:57, 304.49s/it]C:\Users\wz\AppData\Local\Temp\ipykernel_11504\2530387475.py:21: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.cuda.LongTensor(tensor_array), response
C:

C:\Users\wz\AppData\Local\Temp\ipykernel_11504\2530387475.py:21: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.cuda.LongTensor(tensor_array), response
C:\Users\wz\AppData\Local\Temp\ipykernel_11504\2530387475.py:21: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.cuda.LongTensor(tensor_array), response
C:\Users\wz\AppData\Local\Temp\ipykernel_11504\2530387475.py:21: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.cuda.LongTensor(tensor_array), response
C:\Users\wz\AppData\Local\Temp\ipykernel_11504\2530387475.py:21: DeprecationWarning: an integer is required (got type flo

C:\Users\wz\AppData\Local\Temp\ipykernel_11504\2530387475.py:21: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.cuda.LongTensor(tensor_array), response
C:\Users\wz\AppData\Local\Temp\ipykernel_11504\2530387475.py:21: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.cuda.LongTensor(tensor_array), response
C:\Users\wz\AppData\Local\Temp\ipykernel_11504\2530387475.py:21: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.cuda.LongTensor(tensor_array), response
C:\Users\wz\AppData\Local\Temp\ipykernel_11504\2530387475.py:21: DeprecationWarning: an integer is required (got type flo

...

C:\Users\wz\AppData\Local\Temp\ipykernel_11504\2530387475.py:21: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.cuda.LongTensor(tensor_array), response
C:\Users\wz\AppData\Local\Temp\ipykernel_11504\2530387475.py:21: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.cuda.LongTensor(tensor_array), response
C:\Users\wz\AppData\Local\Temp\ipykernel_11504\2530387475.py:21: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.cuda.LongTensor(tensor_array), response
C:\Users\wz\AppData\Local\Temp\ipykernel_11504\2530387475.py:21: DeprecationWarning: an integer is required (got type flo

C:\Users\wz\AppData\Local\Temp\ipykernel_11504\2530387475.py:21: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.cuda.LongTensor(tensor_array), response
C:\Users\wz\AppData\Local\Temp\ipykernel_11504\2530387475.py:21: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.cuda.LongTensor(tensor_array), response
 88%|██████████████████████████████████████████████████████████████████▉         | 88/100 [7:28:04<1:01:01, 305.16s/it]C:\Users\wz\AppData\Local\Temp\ipykernel_11504\2530387475.py:21: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.cuda.LongTensor(tensor_array), response
C:

C:\Users\wz\AppData\Local\Temp\ipykernel_11504\2530387475.py:21: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.cuda.LongTensor(tensor_array), response
C:\Users\wz\AppData\Local\Temp\ipykernel_11504\2530387475.py:21: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.cuda.LongTensor(tensor_array), response
C:\Users\wz\AppData\Local\Temp\ipykernel_11504\2530387475.py:21: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.cuda.LongTensor(tensor_array), response
C:\Users\wz\AppData\Local\Temp\ipykernel_11504\2530387475.py:21: DeprecationWarning: an integer is required (got type flo

C:\Users\wz\AppData\Local\Temp\ipykernel_11504\2530387475.py:21: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.cuda.LongTensor(tensor_array), response
C:\Users\wz\AppData\Local\Temp\ipykernel_11504\2530387475.py:21: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.cuda.LongTensor(tensor_array), response
C:\Users\wz\AppData\Local\Temp\ipykernel_11504\2530387475.py:21: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.cuda.LongTensor(tensor_array), response
C:\Users\wz\AppData\Local\Temp\ipykernel_11504\2530387475.py:21: DeprecationWarning: an integer is required (got type flo

C:\Users\wz\AppData\Local\Temp\ipykernel_11504\2530387475.py:21: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.cuda.LongTensor(tensor_array), response
 98%|████████████████████████████████████████████████████████████████████████████▍ | 98/100 [8:18:57<10:10, 305.39s/it]C:\Users\wz\AppData\Local\Temp\ipykernel_11504\2530387475.py:21: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.cuda.LongTensor(tensor_array), response
C:\Users\wz\AppData\Local\Temp\ipykernel_11504\2530387475.py:21: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.cuda.LongTensor(tensor_array), response
C: